In [1]:
with open("input") as f:
    data = [line.strip() for line in f]

In [2]:
def process_messages():
    while modules_to_send:
        module = modules_to_send.pop(0)
        module.send()

class Module:
    def __repr__(self):
        return f"<{self.__class__.__name__} id={self.id} outputs={', '.join(o.id for o in self.outputs)}>"

class BroadcastModule(Module):
    def __init__(self):
        self.id = "broadcaster"
        self.value = False
        self.outputs = []

    def receive(self, v: bool, *, sender: str):
        global low_pulses_sent
        global high_pulses_sent
        high_pulses_sent += 1 if v else 0
        low_pulses_sent += 0 if v else 1
        self.value = v
        modules_to_send.append(self)

    def send(self):
        for module in self.outputs:
            module.receive(self.value, sender=self.id)

class FlipFlopModule(Module):
    def __init__(self, id: str):
        self.id = id
        self.value = False
        self.outputs = []

    def receive(self, v: bool, *, sender: str):
        global low_pulses_sent
        global high_pulses_sent
        high_pulses_sent += 1 if v else 0
        low_pulses_sent += 0 if v else 1
        if not v:
            self.value = not self.value
            modules_to_send.append(self)

    def send(self):
        for module in self.outputs:
            module.receive(self.value, sender=self.id)

class ConjunctionModule(Module):
    def __init__(self, id: str, *, watched: bool = False):
        self.id = id
        self.value = False
        self.outputs = []
        self.inputs = {}
        self.watched = watched

    def receive(self, v: bool, *, sender: str):
        global low_pulses_sent
        global high_pulses_sent
        high_pulses_sent += 1 if v else 0
        low_pulses_sent += 0 if v else 1
        self.inputs[sender] = v
        modules_to_send.append(self)
        if watching and self.watched and not v and i > 0:
            self.watched = False
            print(self.id, i)

    def send(self):
        v = not all(self.inputs.values())
        for module in self.outputs:
            module.receive(v, sender=self.id)

class OutputModule(Module):
    def __init__(self, id: str):
        self.id = id
        self.value = False
        self.outputs = []

    def receive(self, v: bool, *, sender: str):
        global low_pulses_sent
        global high_pulses_sent
        high_pulses_sent += 1 if v else 0
        low_pulses_sent += 0 if v else 1
        self.value = v

In [3]:
def get_id(line):
    return "broadcaster" if line.startswith("broadcaster") else line[1:].split(" ")[0]

def make_modules() -> dict[str, Module]:
    modules = {}
    for line in data:
        id = get_id(line)
        if line.startswith("broadcaster"):
            modules[id] = BroadcastModule()
        elif line.startswith("%"):
            modules[id] = FlipFlopModule(id)
        elif line.startswith("&"):
            watched = id in {"br", "rz", "lf", "fk"}
            modules[id] = ConjunctionModule(id, watched=watched)

    for line in data:
        id = get_id(line)
        outputs = line.split("-> ")[-1].split(", ")
        for output in outputs:
            if output not in modules:
                modules[output] = OutputModule(output)
            modules[id].outputs.append(modules[output])

    for module in modules.values():
        if type(module) is ConjunctionModule:
            module.inputs = {id: False for id, m in modules.items() if module in m.outputs}

    return modules

def press_the_button(modules):
    modules["broadcaster"].receive(False, sender="button")
    process_messages()


In [4]:
modules_to_send = []
low_pulses_sent = 0
high_pulses_sent = 0
watching = False

modules = make_modules()

for i in range(1000):
    out = press_the_button(modules)

print("Part 1:")
print(low_pulses_sent * high_pulses_sent)

Part 1:
817896682


In [5]:
from graphviz import Digraph

dot = Digraph(format='svg')

def module_name(module):
    c = {
        FlipFlopModule: "%",
        ConjunctionModule: "&",
    }.get(module.__class__, "")
    return c + module.id

nodes = {
    module.id: dot.node(module_name(module))
    for module in modules.values()
}

for module in modules.values():
    for output in module.outputs:
        dot.edge(module_name(module), module_name(output))
dot.render("output")

'output.svg'

In [6]:
modules = make_modules()
watching = True

i = 0
while any(getattr(module, "watched", False) for module in modules.values()):
    i += 1
    out = press_the_button(modules)


br 3877
lf 3911
rz 4057
fk 4079


In [7]:
import math

print("Part 2:")
print(math.lcm(3877, 3911, 4057, 4079))

Part 2:
250924073918341
